# Table of Contents
* [1) Cost Function and Backpropagation](#1%29-Cost-Function-and-Backpropagation)
    * [1) Cost Function](#1%29-Cost-Function)
	* [2) Backpropagation Algorithm](#2%29-Backpropagation-Algorithm)
    * [3) Backpropagation Intuition](#3%29-Backpropagation-Intuition)
* [2) Backpropagation in Practice](#2%29-Backpropagation-in-Practice)
    * [1) Implementation Note: Unrolling Parameters](#1%29-Implementation-Note%3A-Unrolling-Parameters)
	* [2) Gradient Checking](#2%29-Gradient-Checking)
    * [3) Random Initialization](#3%29-Random-Initialization)
    * [4) Putting it together](#2%29-Putting-it-together)
* [3) Application of Neural Networks](#3%29-Application-of-Neural-Networks)
	* [1) Autonomous Driving](#1%29-Autonomous-Driving)	

# 1) Cost Function and Backpropagation

## 1) Cost Function

I'd like to start talking about a learning algorithm for fitting the parameters of a neural network given a training set. As with the discussion of most of our learning algorithms, we're going to begin by talking about the cost function for fitting the parameters of the network. 

I'm going to focus on the application of neural networks to classification problems. So suppose we have a network like that shown on the left. 
- we have a training set like this $(x^{(1)},y^{(1)})$ of m training examples.
- L = total number of layers in this network, so for the network shown on the left we would have L = 4.
- $s_l$ = number of units, that is the number of neurons (not counting the bias unit) in their L of the network. For example: $s_1 = 4, s_2 = 5, s_3 = 5, s_4 = s_L = 4$

We're going to consider two types of classification problems.
- **The first is Binary classification, where the labels y are either 0 or 1. **
    - In this case, we will have 1 output unit, so this Neural Network unit on top has 4 output units, but if we had binary classification we would have only one output unit that computes $h_{\Theta}x$. 
    - And the output of the neural network of $h_{\Theta}x$ is going to be a real number, so $h_{\Theta}x \in R$. 
    - And in this case the number of output units, $S_L$, where L is again the index of the final layer. Cuz that's the number of layers we have in the network so the number of units we have in the output layer is going to be equal to 1. In this case to simplify notation later, I'm also going to set K=1 so you can think of K as also denoting the number of units in the output layer. 
- **The second type of classification problem we'll consider will be multi-class classification problem where we may have K distinct classes.**
    - So our early example had this representation for y, e.g: $\begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \end{bmatrix}$, $\begin{bmatrix} 0 \\ 1 \\ 0 \\ 0 \end{bmatrix}$, $\begin{bmatrix} 0 \\ 0 \\ 1 \\ 0 \end{bmatrix}$, $\begin{bmatrix} 0 \\ 0 \\ 0 \\ 1 \end{bmatrix}$, if we have 4 classes, and in this case we will have capital K output units and our hypothesis or output vectors that are K dimensional. 
    - And the number of output units will be equal to K, so $S_L = K$. And usually we would have K $\geq$ 3 in this case, because if we had two causes, then we don't need to use the one verses all method. We use the one verses all method only if we have K greater than or equals V classes, so having only two classes we will need to use only one upper unit. 

<img src="images/lec9_pic01.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/na28E/cost-function) 2:00*

<!--TEASER_END-->

**Now let's define the cost function for our neural network. **

The cost function we use for the neural network is going to be a generalization of the one that we use for logistic regression. For logistic regression we used to minimize the cost function $J(\theta)$ that was $-\frac{1}{m}$ of this cost function and
then plus this extra regularization term here, where this was a sum from J=1 through n, because we did not regularize the bias term $\theta_0$. 

For a neural network, our cost function is going to be a generalization of this. Where instead of having basically just one, which is the compression output unit, we may instead have K of them. 

So here's our cost function. 
$$J(\Theta) = -\frac{1}{m} \bigg[\sum_{i=1}^m \sum_{k=1}^K y_k^{(i)} \log (h_\Theta(x^{(i)}))_k + (1 - y_k^{(i)}\log(1 - (h_\Theta(x^{(i)}))_k) \bigg] + \frac{\lambda}{2m}\sum_{l=1}^{L-1}\sum_{i=1}^{s_l}\sum_{j=1}^{s_{l+1}}(\Theta_{ji}^{(l)})^2$$

Our new network now outputs vectors in $R^K$, so $h_\Theta(x) \in R^K$, where R might be equal to 1 if we have a binary classification problem. 
- I'm going to use this notation $(h_\Theta(x))_i$ to denote the ith output. That is, $h_\Theta(x)$ is a k-dimensional vector and so this subscript i just selects out the ith element of the vector that is output by my neural network. 

My cost function $J(\Theta)$ is now going to be the following. 
- $-\frac{1}{m}$ of a similar term to what we have for logistic regression, except that we have the sum from K = 1 through K. This summation is basically a sum over my K output. So if I have four output units, that is if the final layer of my neural network has four output units, then this is a sum from k =1 through four of basically the logistic regression algorithm's cost function but summing that cost function over each of my four output units in turn. And so you notice in particular that this applies to $y_k, (h_\Theta(x^{(i)}))_k$, because we're basically taking the K upper  units, and comparing that to the value of $y_k$ which is that one of those vectors saying what cost it should be, for example: $y_k = \begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \end{bmatrix}$
- And finally, **the second term here is the regularization term**, similar to what we had for the logistic regression. This summation term looks really complicated, but all it's doing is it's summing over these terms $\Theta_{ji}^{(l)}$ for all values of i, j and l. 
    - Except that we don't sum over the terms corresponding to these bias values, $\Theta_0, $like we have for logistic regression. Completely, we don't sum over the terms responding to where i = 0. So that is because when we're computing the activation of a neuron, we have terms like these $\Theta_{i0}^{(2)}x_0 + \Theta_{i1}^{(2)}x_1 + ...$. And so the values with a zero there, that corresponds to something that multiplies into an $x_0$ or an $a_0$. And so this is kinda like a bias unit and by analogy to what we were doing for logistic regression, we won't sum over those terms in our regularization term because we don't want to regularize them and string their values as zero. 
    - But this is just one possible convention, and even if you were to sum over i = 0 up to $S_l$, it would work about the same and doesn't make a big difference. But maybe this convention of not regularizing the bias term is just slightly more common. 

<img src="images/lec9_pic02.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/na28E/cost-function) 5:00*

<!--TEASER_END-->

<img src="images/lec9_pic03.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/na28E/cost-function) 6:32*

<!--TEASER_END-->

## 2) Backpropagation Algorithm

Let's start to talk about an algorithm, for trying to minimize the cost function. In particular, we'll talk about the **back propagation algorithm**.

Here's the cost function that we wrote down
$$J(\Theta) = -\frac{1}{m} \bigg[\sum_{i=1}^m \sum_{k=1}^K y_k^{(i)} \log (h_\Theta(x^{(i)}))_k + (1 - y_k^{(i)}\log(1 - (h_\Theta(x^{(i)}))_k) \bigg] + \frac{\lambda}{2m}\sum_{l=1}^{L-1}\sum_{i=1}^{s_l}\sum_{j=1}^{s_{l+1}}(\Theta_{ji}^{(l)})^2$$

What we'd like to do is try to find parameters $\Theta$ to try to minimize $J(\Theta)$. In order to use either gradient descent or one of the advance optimization algorithms. What we need to do therefore is to write code that takes this input the parameters $\Theta$ and computes 
- $J(\Theta)$
- and these partial derivative terms $\frac{\partial}{\partial \Theta_{ij}^{(l)}}J(\Theta)$.

Remember, that the parameters in the the neural network of these things, $\Theta_{ij}^{(l)}$, that's the real number, so $\Theta_{ij}^{(l)} \in R$

In order to compute the cost function $J(\Theta)$, we just use this formula up here and so, what I want to do is focus on talking about how we can compute these partial derivative terms $\frac{\partial}{\partial \Theta_{ij}^{(l)}}J(\Theta)$.

<img src="images/lec9_pic04.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/1z9WW/backpropagation-algorithm) 0:32*

<!--TEASER_END-->

Let's start by talking about the case of when we have only one training example, which is a pair $(x, y)$. Let's tap through the sequence of calculations we would do with this one training example.

The first thing we do is we apply forward propagation in order to compute whether a hypotheses actually outputs given the input x.
- $a^{(1)}$: the activation values of this first layer
- $a^{(2)} = g(z^{(2)})$: g is the sigmoid activation function applied to $z^{(2)$ and this would give us our activations for the first middle layer, we also add $a_0^{(2)}$ which is the bias term.
- Next we apply 2 more steps of this forward propagation to compute $a^{(3)}$ and $a^{(4)}$ where $a^{(4)}$ is also the output of a hypotheses h of x.

So this is our vectorized implementation of forward propagation and it allows us to compute the activation values for all of the neurons in our neural network. 

Next, in order to compute the derivatives, we're going to use an algorithm called back propagation. 

<img src="images/lec9_pic05.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/1z9WW/backpropagation-algorithm) 1:22*

<!--TEASER_END-->

The intuition of the back propagation algorithm is that for each note we're going to compute the term $\delta_j^{(l)}$, delta superscript l subscript j, that's going to somehow represent the error of note j in the layer l. 
- Recall $a_j^{(l)}$: that does the activation of the j of unit in layer l.

This delta term is in some sense going to capture our error in the activation of that neural duo. So, how we might wish the activation of that note is slightly different.

Concretely, taking the example neural network that we have on the right which has four layers. 
- L = 4. For each output unit, we're going to compute this delta term. So, delta for the j of unit in the fourth layer is equal to just the activation of that unit minus what was the actual value of 0 in our training example.
$$\delta_j^{(4)} = a_j^{(4)} - y_j$$
- Because $a_j^{(4)} = (h_\Theta(x))_j$, so this delta term is just the difference between when a hypotheses output and what was the value of y in our training set whereas $y_j$ is the j of element of the vector value y in our labeled training set. 
- And if you think of $\delta$, a and y as vectors then you can also take those and come up with a vectorized implementation of it, which is just $\delta^{(4)} = a^{(4)} - y$. Where here, each of these $\delta^{(4)}$, $a^{(4)}$, and y, each of these is a vector whose dimension is equal to the number of output units in our network.

So we've now computed the error term $\delta^{(4)}$ for our network. **What we do next is compute the delta terms for the earlier layers in our network. **
$$\delta_j^{(3)} = (\Theta^{(3)})^T\delta_j^{(4)} .* g'(z^{(3)})$$

- $(\Theta^{(3)})^T\delta_j^{(4)}$: this is a vector
- $g'(z^{(3)})$: this is also a vector. This term $g'(z^{(3)})$, that formally is actually the derivative of the activation function g evaluated at the input values given by z3. If you know calculus, you can try to work it out yourself and see that you can simplify it to the same answer that I get. But I'll just tell you pragmatically what that means. What you do to compute this g prime, these derivative terms is just $a^{(3)} .* (1 - a^{(3)})$ where $a^{(3)}$ is the vector of activations. 1 is the vector of ones and $a^{(3)}$ is again the activation the vector of activation values for that layer. 
- The $.*$ this dot times, this is the element y's multiplication operation that we know from MATLAB. So in this case, it is element wise multiplication between these two vectors.

**Next you apply a similar formula to compute delta 2**
$$\delta_j^{(2)} = (\Theta^{(2)})^T\delta_j^{(3)} .* g'(z^{(2)})$$



<img src="images/lec9_pic07.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/1z9WW/backpropagation-algorithm) 5:00*

<!--TEASER_END-->

<img src="images/lec9_pic08.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/1z9WW/backpropagation-algorithm) 8:00*

<!--TEASER_END-->

<img src="images/lec9_pic06.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/1z9WW/backpropagation-algorithm) 11:27*

<!--TEASER_END-->

## 3) Backpropagation Intuition

<img src="images/lec9_pic09.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/du981/backpropagation-intuition) 1:00*

<!--TEASER_END-->

<img src="images/lec9_pic10.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/du981/backpropagation-intuition) 2:30*

<!--TEASER_END-->

<img src="images/lec9_pic11.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/du981/backpropagation-intuition) 5:00*

<!--TEASER_END-->

<img src="images/lec9_pic12.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/du981/backpropagation-intuition) 7:00*

<!--TEASER_END-->

<img src="images/lec9_pic13.png">
<img src="images/lec9_pic14.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/du981/backpropagation-intuition) 12:11*

<!--TEASER_END-->

# 2) Backpropagation in Practice

## 1) Implementation Note: Unrolling Parameters

<img src="images/lec9_pic15.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/60Uxp/implementation-note-unrolling-parameters) 1:00*

<!--TEASER_END-->

<img src="images/lec9_pic16.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/60Uxp/implementation-note-unrolling-parameters) 2:00*

<!--TEASER_END-->

<img src="images/lec9_pic18.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/60Uxp/implementation-note-unrolling-parameters) 3:47*

<!--TEASER_END-->

<img src="images/lec9_pic17.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/60Uxp/implementation-note-unrolling-parameters) 5:43*

<!--TEASER_END-->

## 2) Gradient Checking

<img src="images/lec9_pic19.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/Y3s6r/gradient-checking) 2:00*

<!--TEASER_END-->

<img src="images/lec9_pic21.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/Y3s6r/gradient-checking) 5:01*

<!--TEASER_END-->

<img src="images/lec9_pic20.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/Y3s6r/gradient-checking) 5:42*

<!--TEASER_END-->

<img src="images/lec9_pic22.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/Y3s6r/gradient-checking) 6:30*

<!--TEASER_END-->

<img src="images/lec9_pic23.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/Y3s6r/gradient-checking) 9:00*

<!--TEASER_END-->

<img src="images/lec9_pic24.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/Y3s6r/gradient-checking) 11:19*

<!--TEASER_END-->

## 3) Random Initialization

<img src="images/lec9_pic25.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/ND5G5/random-initialization) 0:20*

<!--TEASER_END-->

<img src="images/lec9_pic26.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/ND5G5/random-initialization) 1:20*

<!--TEASER_END-->

<img src="images/lec9_pic27.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/ND5G5/random-initialization) 4:45*

<!--TEASER_END-->

<img src="images/lec9_pic28.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/ND5G5/random-initialization) 6:15*

<!--TEASER_END-->

## 4) Putting it together

<img src="images/lec9_pic29.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/Wh6s3/putting-it-together) 0:30*

<!--TEASER_END-->

<img src="images/lec9_pic30.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/Wh6s3/putting-it-together) 4:30*

<!--TEASER_END-->

<img src="images/lec9_pic31.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/Wh6s3/putting-it-together) 7:30*

<!--TEASER_END-->

<img src="images/lec9_pic32.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/Wh6s3/putting-it-together) 9:30*

<!--TEASER_END-->

<img src="images/lec9_pic33.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/Wh6s3/putting-it-together) 11:58*

<!--TEASER_END-->

# 3) Application of Neural Networks

## 1) Autonomous Driving

<img src="images/lec9_pic34.png">

*Screenshot taken from [Coursera](https://www.coursera.org/learn/machine-learning/lecture/Wh6s3/putting-it-together) 3:00*

<!--TEASER_END-->